In [ ]:
### openweather API

import requests
import mysql.connector
from mysql.connector import Error
from datetime import datetime, timezone
import schedule
import time

# OpenWeatherMap API 설정
OPENWEATHER_API_KEY = "12cbb94c69458b0c45a01c760c2e7bf1"
OPENWEATHER_API_URL = "https://api.openweathermap.org/data/2.5/forecast"

# MySQL 데이터베이스 설정
MYSQL_HOST = 'localhost'
MYSQL_USER = 'root'
MYSQL_PASSWORD = '0000'
MYSQL_DATABASE = 'news_database'

# 거제시의 위도와 경도
latitude = 34.8806
longitude = 128.6211

# MySQL에 날씨 데이터 저장 함수
def store_weather_data(data):
    # MySQL 서버에 연결
    connection = mysql.connector.connect(
        host = MYSQL_HOST,
        user = MYSQL_USER,
        password = MYSQL_PASSWORD,
        database = MYSQL_DATABASE
    )
    cursor = connection.cursor()

    insert_query = """
    INSERT INTO weather (dt, temperature, max_temperature, min_temperature, humidity, wind_power, weather_description) VALUES (%s, %s, %s, %s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE
    temperature = VALUES(temperature), max_temperature = VALUES(max_temperature), min_temperature = VALUES(min_temperature), humidity = VALUES(humidity), wind_power = VALUES(wind_power), weather_description = VALUES(weather_description)
    """
            
    for forecast in data['list']:
        timestamp = forecast['dt']
        dt = datetime.fromtimestamp(timestamp, timezone.utc)
        temperature = forecast['main']['temp']
        max_temperature = forecast['main']['temp_max']
        min_temperature = forecast['main']['temp_min']
        humidity = forecast['main']['humidity']
        wind_power = forecast['wind']['speed']
        weather_description = forecast['weather'][0]['description']
        
        # 전달할 값이 7개로 쿼리와 맞추어야 함
        document = (dt, temperature, max_temperature, min_temperature, humidity, wind_power, weather_description)
        cursor.execute(insert_query, document)

    connection.commit()
    cursor.close()
    connection.close()


# 날씨 데이터를 가져오는 함수
def fetch_weather_data():
    params = {
        "lat": latitude,    
        "lon": longitude,   
        "appid": OPENWEATHER_API_KEY,
        "units": "metric",   
        "lang": "kr" 
    }
    

    response = requests.get(OPENWEATHER_API_URL, params=params)
    response.raise_for_status()
    data = response.json()
    
    store_weather_data(data)
    
def main():
    print("초기 데이터 가져오기...")
    fetch_weather_data()
    
    schedule.every(3).hours.do(fetch_weather_data)
    
    print("스케줄러가 시작되었습니다.")
    try:
        while True:
            schedule.run_pending()
            time.sleep(1)
    except KeyboardInterrupt:
        print("스케줄러가 중지되었습니다.")

if __name__ == "__main__":
    main()